## Cover Song Identification using comparison of hierarchical structure for covers80

### Library Importing

In [14]:
import librosa
import numpy as np
import scipy
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp2d
from scipy.sparse.csgraph import laplacian
from scipy.spatial.distance import directed_hausdorff
from scipy.cluster import hierarchy
from scipy.linalg import eigh
from scipy.ndimage import median_filter
import cv2
from sklearn import metrics
import matplotlib.pyplot as plt
import dill
import sys
import glob
import os
import random

### Dill session

In [ ]:
dill.dump_session('sets.db')

In [ ]:
dill.load_session('sets.db')

### Load audio

In [23]:
#Choose directory containing audiofiles
directory = '../../../Datasets/covers80/covers32k'

In [26]:
#Read all paths in specified directory
all_dirs = []
all_roots = []
all_names= []
for root, dirs, files in os.walk(directory):
        for name in files:
            if (('.wav' in name) or ('.aif' in name) or ('.mp3' in name)):
                filepath = os.path.join(root, name)
                all_dirs.append(filepath)
                all_names.append(name[:-4])
                all_roots.append(root)
file_no = len(all_names)
#Load all audiofiles and store in array
all_audio = []
for f in range(file_no):
    y, sr = librosa.load(all_dirs[f], sr=16000, mono=True)
    all_audio.append((y, sr))
    sys.stdout.write("\rLoaded %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()

  warnings.warn("PySoundFile failed. Trying audioread instead.")
Loaded 73/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Loaded 74/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Loaded 75/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Loaded 76/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Loaded 77/160 piece

### Hierarchical structure decomposition

In [31]:
#CQT config
BINS_PER_OCTAVE = 12 
N_OCTAVES = 5

#Approximations
kmin = 2
kmax = 8

all_struct = []

for i in range(file_no):
    y, sr = all_audio[i]
   
    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y, sr=sr,
                                        bins_per_octave=BINS_PER_OCTAVE,
                                        n_bins=N_OCTAVES * BINS_PER_OCTAVE)),
                            ref=np.max)
    
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)

    Csync = librosa.util.sync(C, beats, aggregate=np.median)

    Cstack = librosa.feature.stack_memory(Csync, 4)
    
    R = librosa.segment.recurrence_matrix(Cstack, width=3, mode='affinity', sym=True)

    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    Rf = librosa.segment.path_enhance(Rf, 15)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    Msync = librosa.util.sync(mfcc, beats)

    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)

    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)

    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)

    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)

    A = mu * Rf + (1 - mu) * R_path
    
    L = scipy.sparse.csgraph.laplacian(A, normed=True)
    
    #eigendecomposition
    evals, evecs = scipy.linalg.eigh(L)
    #eigenvector filtering
    evecs = scipy.ndimage.median_filter(evecs, size=(9, 1))
    #normalization
    Cnorm = np.cumsum(evecs**2, axis=1)**0.5
    dist_set = []
    for k in range(kmin, kmax):
        Xs = evecs[:, :k] / Cnorm[:, k-1:k]
        distance = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(Xs, metric='euclidean'))
        dist_set.append(distance)
    all_struct.append(dist_set)    

    sys.stdout.write("\rComputed for %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()

Computed for 160/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
Computed for 160/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
Computed for 160/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
Computed for 160/160 pieces./Users/chris/Library/Python/3.8/lib/python/site-packages/librosa/filters.py:238: UserWa

KeyboardInterrupt: 

### Flatten upper triangle of each approximation

In [22]:
all_flat = []
for i in range(file_no):
    all_flat.append([])
    for j in range(kmax-kmin):
        all_flat[i].append((all_struct[i][j])[np.triu_indices(all_struct[i][j].shape[0])])
    sys.stdout.write("\rComputed for %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()  
    
all_flat = np.asarray(all_flat)
print(all_flat.shape)

Computed for 166/166 pieces.

IndexError: list index out of range

### Concatenate hierarchies to single vector

In [ ]:
all_merged = []
for i in range(file_no):
    all_merged.append(np.empty((0)))
    for j in range(kmax-kmin):
        all_merged[i] = np.concatenate((all_merged[i], all_flat[i][j]))
    sys.stdout.write("\rComputed for %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()
    
all_merged = np.asarray(all_merged)
print(all_merged.shape)

### Boolean, pairwise matrix for cover (True) vs non-cover (False)

In [ ]:
#True if cover, False if non-cover
covers = np.zeros(file_no, file_no, dtype=np.bool_)
for i in range(file_no):
    for j in range(file_no):
        if (all_roots[i] == all_roots[j]):
            covers[i][j] = True
        else:
            covers[i][j] = False
plt.matshow(covers)
plt.show()

### L1 Distances

In [ ]:
L1_distances_covers = []
L1_distances_noncovers = []
for i in range(all_flat.shape[0]):
    for j in range(all_flat.shape[0]):
        if covers[i][j]:
            if (L1_distances[i][j] != 0):
                L1_distances_covers.append(L1_distances[i][j])
        else:
            L1_distances_noncovers.append(L1_distances[i][j])
            
plt.figure()
plt.hist(L1_distances_covers, bins=200, alpha=0.5, label='Covers', density=1)
plt.hist(L1_distances_noncovers, bins=200, alpha=0.5, label='Non-covers', density=1)
plt.title("Histogram of L1 distances between cover and non-cover pairs")
plt.legend(loc='upper right')
plt.show()